In [ ]:
import sys

sys.path.append('../../')
import json
import torchaudio

import librosa
import torch
from academicodec.models.hificodec.vqvae import VQVAE
from academicodec.models.hificodec.models import Encoder
from librosa.util import normalize

In [8]:
config_path = './config_24k_960d.json'
with open(config_path, 'r') as f:
    config = json.load(f)
    sample_rate = config['sampling_rate']

In [9]:
print("Init model and load weights")
model = VQVAE(
    config_path,
    ckpt_path="",
    with_encoder=True)
# model.cuda()
model.eval()
print("Model ready")

Init model and load weights


Model ready


In [10]:
wav_path = './sample.wav'

wav, sr = librosa.load(wav_path, sr=sample_rate)
print("wav.shape:",wav.shape)
assert sr == sample_rate

wav = normalize(wav) * 0.95
wav = torch.FloatTensor(wav).unsqueeze(0)
# wav = wav.to(torch.device('cuda'))

wav.shape: (240000,)


In [11]:
acoustic_token = model.encode(wav)

print("acoustic_token:", "\n",acoustic_token)
print("acoustic_token.shape:",acoustic_token.shape)
print("acoustic_token.dtype:",acoustic_token.dtype)

acoustic_token: 
 tensor([[[3550, 3080],
         [3550, 1232],
         [3550, 1232],
         [3550, 1232],
         [3550, 1232],
         [3550, 1232],
         [3550, 1232],
         [3550,  475],
         [3550, 1232],
         [3550, 3778],
         [3550, 1232],
         [3550, 1232],
         [3550, 1232],
         [3550, 1232],
         [3550, 1232],
         [3550,  475],
         [ 159, 1232],
         [1084,  202],
         [2470,  475],
         [1084,  475],
         [3550, 1232],
         [3550, 1232],
         [3550, 1232],
         [3550, 1232],
         [3550, 3778],
         [3550, 1533],
         [3550, 1232],
         [3550, 1232],
         [3550, 1232],
         [3550, 1232],
         [3550, 1232],
         [3550, 1232],
         [3550, 1232],
         [3550, 1232],
         [3550, 1232],
         [3550, 1232],
         [3550, 1232],
         [3550, 1232],
         [3550, 1232],
         [3550, 1232],
         [3550, 1232],
         [3550, 1232],
         [3550, 

In [12]:
quant_vector = model.encoder(wav.unsqueeze(1))
print(quant_vector.shape)
y_hat = model.generator(quant_vector)
print(y_hat.shape)

torch.Size([1, 128, 250])
torch.Size([1, 1, 240000])


In [13]:
y_hat = y_hat.squeeze(0)
y_hat = y_hat.cpu()
print(y_hat.size())
torchaudio.save("y_hat.wav", torch.tensor(y_hat), sample_rate, channels_first=True)

torch.Size([1, 240000])


/tmp/ipykernel_152416/3900361264.py:4: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  torchaudio.save("y_hat.wav", torch.tensor(y_hat), sample_rate, channels_first=True)


: 